In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [ ]:
train_labels.shape, test_labels.shape

((60000,), (10000,))

In [ ]:
type(train_labels), type(test_labels)


(numpy.ndarray, numpy.ndarray)

In [ ]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")      # 10 es el Outshape (son 10 digitos los que queremos predecir)
])


# An optimizer—The mechanism through which the model will update itself based on the training data it sees, so as to improve its performance.
# A loss function—How the model will be able to measure its performance on the training data, and thus how it will be able to steer itself in the right direction.
# Metrics to monitor during training and testing—Here, we’ll only care about accu-racy (the fraction of the images that were correctly classified).

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


# Before training, we’ll preprocess the data by reshaping it into the shape the model expects and scaling it so that all values are in the [0, 1] interval.
# Previously, our training images were stored in an array of shape (60000, 28, 28) of type uint8 with values in the [0, 255] interval.
# We’ll transform it into a float32 array of shape (60000, 28 * 28) with values between 0 and 1. uwedan como(60000, 784)

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

# Fitting the model

model.fit(train_images, train_labels, epochs=5, batch_size=128)



Epoch 1/5
469/469 [==============================] - 8s 13ms/step - loss: 0.2632 - accuracy: 0.9238
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1066 - accuracy: 0.9684
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0711 - accuracy: 0.9790
Epoch 4/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0509 - accuracy: 0.9848
Epoch 5/5
469/469 [==============================] - 4s 9ms/step - loss: 0.0378 - accuracy: 0.9886


In [ ]:
# Make predictions

test_digits = test_images[0:10]
test_labels[0:10]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=uint8)

In [ ]:
predictions = model.predict(test_digits)

for i in range (0, 10):
  print(predictions[i].argmax())

1/1 [==============================] - 0s 78ms/step
7
2
1
0
4
1
4
9
5
9


In [ ]:
# Evaluate the model

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")




313/313 [==============================] - 1s 4ms/step - loss: 0.0708 - accuracy: 0.9779
test_acc: 0.9779000282287598


In [ ]:
# Definicion de una metrica a medida
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):
  def __init__(self, name="rmseDiego", **kwargs):
    super().__init__(name=name, **kwargs)
    self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
    self.total_samples = self.add_weight(name="total_samples", initializer="zeros", dtype="int32")

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
    mse = tf.reduce_sum(tf.square(y_true - y_pred))
    self.mse_sum.assign_add(mse)
    num_samples = tf.shape(y_pred)[0]
    self.total_samples.assign_add(num_samples)

  def result(self):
    return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

  def reset_state(self):
    self.mse_sum.assign(0.)
    self.total_samples.assign(0)


In [ ]:


model.compile(optimizer="rmsprop",
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy", RootMeanSquaredError()])

model.fit(train_images, train_labels,epochs=3,
          validation_data=(test_images, test_labels))

test_metrics = model.evaluate(test_images, test_labels)


Epoch 1/3
1875/1875 [==============================] - 15s 7ms/step - loss: 0.0448 - accuracy: 0.9858 - rmseDiego: 7.4244 - val_loss: 0.0729 - val_accuracy: 0.9784 - val_rmseDiego: 7.4481
Epoch 2/3
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0351 - accuracy: 0.9890 - rmseDiego: 7.4379 - val_loss: 0.0750 - val_accuracy: 0.9781 - val_rmseDiego: 7.4515
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - loss: 0.0733 - accuracy: 0.9801 - rmseDiego: 7.4578
